In [0]:
import tensorflow as tf
import numpy as np
from tqdm import tqdm

In [46]:
!pip install tqdm

In [4]:
from google.colab import files
uploaded = files.upload()

Saving C-code.zip to C-code.zip


In [5]:
!unzip C-code.zip

Archive:  C-code.zip
   creating: C-code/
  inflating: C-code/ac.c             
  inflating: C-code/aio.c            
  inflating: C-code/bcd.c            
  inflating: C-code/bch.c            
  inflating: C-code/br.c             
  inflating: C-code/bug.c            
  inflating: C-code/cma.c            
  inflating: C-code/dax.c            
  inflating: C-code/dd.c             
  inflating: C-code/dh.c             
  inflating: C-code/dm.c             
  inflating: C-code/ea.c             
  inflating: C-code/ec.c             
  inflating: C-code/fc.c             
  inflating: C-code/fd.c             
  inflating: C-code/fdt.c            
  inflating: C-code/fs.c             
  inflating: C-code/fs2.c            
  inflating: C-code/gc.c             
  inflating: C-code/gc2.c            
  inflating: C-code/gc3.c            
  inflating: C-code/gcd.c            
  inflating: C-code/gup.c            
  inflating: C-code/gw.c             
  inflating: C-code/he.c             
  inflat

C-code	C-code.zip  datalab  testdata.txt  training.txt


In [0]:
import os
text = ""
for fname in os.listdir("C-code"):
    with open(os.path.join("C-code", fname), "r") as f:
        text += f.read() + "\n"

vocab = set(text)
vocab_to_int = dict()
int_to_vocab = dict()
for i, c in enumerate(vocab):
  vocab_to_int[c] = i
  int_to_vocab[i] = c

enc_text = [vocab_to_int[c] for c in text]

In [0]:
seq_length = 100
sequences = []
out_sequences = []
for i in range(0, len(enc_text), seq_length):
  sequences += [enc_text[i: i+seq_length]]
  out_sequences += [enc_text[i+1: i+seq_length+1]]

sequences = np.array(sequences[:-1], np.int32)
out_sequences = np.array(out_sequences[:-1], np.int32)


In [0]:
tf.reset_default_graph()
inputs = tf.placeholder(tf.int32, [None, seq_length])
targets = tf.placeholder(tf.int32, [None, seq_length])

In [0]:
drop = 0.5
lstm1 = tf.contrib.rnn.BasicLSTMCell(512)
layer1 = tf.contrib.rnn.DropoutWrapper(lstm1, output_keep_prob=drop)

lstm2 = tf.contrib.rnn.BasicLSTMCell(512)
layer2 = tf.contrib.rnn.DropoutWrapper(lstm2, output_keep_prob=drop)

In [0]:

cell = tf.contrib.rnn.MultiRNNCell([layer1])
initial_state =cell.zero_state(200, tf.float32)

x_one_hot = tf.one_hot(inputs, len(vocab))
out, out_state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=initial_state)

In [0]:
out_flat = tf.reshape(out, [-1, 512])

w = tf.Variable(tf.truncated_normal(shape=(512, len(vocab)), stddev=0.1))
b = tf.Variable(tf.zeros(len(vocab)))

logits = tf.add(tf.matmul(out_flat , w), b)
prob = tf.nn.softmax(logits)

In [0]:
y_one_hot = tf.one_hot(targets, len(vocab))
y_reshaped = tf.reshape(y_one_hot, logits.get_shape())

loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
loss = tf.reduce_mean(loss)

In [0]:
adam = tf.train.AdamOptimizer(0.0005)
train_step = adam.minimize(loss)

In [156]:
num_epochs = 50
bat_size = 200
curr = 0
losses = []
sess= tf.Session()
sess.run(tf.global_variables_initializer())
for i in range(num_epochs):
  print("Epoch -", i+1)
  l= s =0
  curr = 0
  for _ in tqdm(range(int(len(sequences)/bat_size)-1)):
    inp = sequences[curr:curr+bat_size]
    tar = out_sequences[curr:curr+bat_size]
    feed_dict = {inputs:inp, targets:tar}
    l, s = sess.run([loss, train_step], feed_dict=feed_dict)
    losses.append(l)
    curr = curr+bat_size
  print('\nLoss: ', l)

  0%|          | 0/61 [00:00<?, ?it/s]

('Epoch -', 1)


  0%|          | 0/61 [00:00<?, ?it/s]

('\nLoss: ', 3.7922304)
('Epoch -', 2)


  0%|          | 0/61 [00:00<?, ?it/s]

...

('\nLoss: ', 1.6280508)
('Epoch -', 49)


  0%|          | 0/61 [00:00<?, ?it/s]

('\nLoss: ', 1.6174848)
('Epoch -', 50)


100%|██████████| 61/61 [00:14<00:00,  4.19it/s]

('\nLoss: ', 1.6049111)


In [157]:
seed = text[:100]
code = ''
enc = [[vocab_to_int[x] for x in seed]]
enc = [enc_text[:100]]*200
#let diversity = 1
code_len = 1000

for _ in tqdm(range(code_len)):
  p = sess.run([prob], feed_dict={inputs:enc})
  idx = np.random.choice(list(range(len(vocab))), p = p[0][-1])
  code += int_to_vocab[idx]
  enc[0] = enc[0][1:] + [idx]
  enc = [enc[0]]*200
  
print(seed+code)

100%|██████████| 1000/1000 [01:46<00:00,  9.42it/s]

#include <linux/bcd.h>
#include <linux/export.h>

unsigned _bcd2bin(unsigned char val)
{
	return (vaid_riapy(cbl)) {
			hm_wert(ho_draster_infs, r);

			}
			 * 
sa il dave by ry nom teine or f2pend to PATTI Srinel  wathanen bo flot
an syquendq.'
			 */
			sem_crine +164;
}

	sif_urch_in_ore_ut(resd_ms, &vest);

		he_writel_herd(he_dev, masb.h)
 * VATjiob[a2].o rouceluede.. Thy mpgarteb-uset io @bchmand enst lectum the llagk uns oened  dorimst botadle. the pernans, wher sware ano
		 * U ermarem < 0 izeod d.s the ous[ill raled -Fsy.
		 * subrectfor ateperse accishen we thp resysb ie the te the id thing thin thi/
 */pentionq].and
		 * mad_vory: idb_tree_CON(Thi wilk pate ufrer ils, nod takg */
		GUN_ge = atast;

			pfr_fise + (\ne - 1;
					hRPT_RNRe(hec));
				hirra[i].sestino = fioddlte_d mede;
					->lac_lock = free;
					_datak_dos[iM] >is;
			} \lM_00x	 fr errgwee .ones.k inderak_punelter		 *faul(wald paret */
				cMubleckur = -ENODEN;

					ofs thj = mpd_lock(is);
					rsti_if (ins

Training for over 10000 epochs can generate realistic looking code. I have trained for only 50 epochs due to the lack of compute available.